<a href="https://colab.research.google.com/github/rodrigorcarmo/multi_agent_chatbot/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets evaluate rouge_score nltk

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b751f89d75ceacf58ae225d198bfc14428b213ade46a6097bbf10ea89aafe930
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# Summarization

This notebook performs a fine tuning on the t5-small model available from the HuggingFace Hub, it performs the same steps as instructed on their website and the purpose was to get an overview and study their capabilities.

In [2]:
# Mounting the Google Drive to access the customer feedback dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Logging on the Hugging Face Hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

Then take a look at an example:

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThis act shall be known, and may be cited, as the Pool Safety Act.\nSEC. 2.\nThe Legislature finds and declares all of the following:\n(a) Swimming pools provide children and their families with a wonderful opportunity for recreation, exercise, and fun. Keeping children safe during this activity is supported by parents and guardians, safety advocates, health providers, insurance companies, and the swimming pool industry.\n(b) According to both the federal Centers for Disease Control and Prevention’s National Center for Injury Prevention and Control and the State Department of Public Health’s EpiCenter data, drowning is the leading cause of death for California children one to four years of age, inclusive.\n(c) Additional children suffer near-drowning incidents and survive, but many of those children suffer irreversible brain injuries, which can lead to lifelong learning deficiencies that impact not on

## Preprocess

The next step is to load a T5 tokenizer to process `text` and `summary`:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="billsum_t5-model_summarization",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.506792,0.150600,0.057500,0.123300,0.123200,19.000000
2,No log,2.449792,0.164400,0.067100,0.135900,0.135700,19.000000
3,No log,2.420736,0.183600,0.082600,0.152100,0.152200,19.000000
4,No log,2.405171,0.188900,0.089600,0.157400,0.157300,19.000000
5,No log,2.399938,0.189600,0.090700,0.158200,0.158100,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=310, training_loss=2.6301684964087704, metrics={'train_runtime': 349.6236, 'train_samples_per_second': 14.144, 'train_steps_per_second': 0.887, 'total_flos': 1338530416558080.0, 'train_loss': 2.6301684964087704, 'epoch': 5.0})

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1728933596.e935b70787ed.2230.2:   0%|          | 0.00/8.91k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rodrigorcarmo/billsum_t5-model_summarization/commit/b58f81b0f1b91bf32cc4495f86f011cc26be3756', commit_message='End of training', commit_description='', oid='b58f81b0f1b91bf32cc4495f86f011cc26be3756', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
from transformers import pipeline
import pandas as pd

summarizer = pipeline("summarization", model="rodrigorcarmo/billsum_t5-model_summarization",device_map="auto",)
tokenizer_kwargs = {'truncation':True,'max_length':512}

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Customer_Feedback_Dataset.csv',sep=';')

positive_texs = df[df['sentiment'] == 'positive']['feedback_text'].tolist()
positive = 'summarize: '+' '.join(positive_texs)
negative_texs = df[df['sentiment'] == 'negative']['feedback_text'].tolist()
negative = 'summarize: '+' '.join(negative_texs)

In [51]:
negative

'summarize: Great product, but the delivery was late. Excellent value for money! Received a defective item. Easy to use website and quick checkout. The delivery was fast and the product is good. Great product, but the delivery was late. Not satisfied with the service. Excellent value for money! Not satisfied with the service. Excellent value for money! Excellent value for money! I had issues with the website. Great product, but the delivery was late. The customer service was very helpful. The pricing is too high for what you get. Not satisfied with the service. Excellent value for money! Easy to use website and quick checkout. Great product, but the delivery was late. Not satisfied with the service. The delivery was fast and the product is good. The customer service was very helpful. The delivery was fast and the product is good. I had issues with the website. The delivery was fast and the product is good. Great product, but the delivery was late. Not satisfied with the service. Receiv

In [3]:
summarizer(negative,**tokenizer_kwargs)

NameError: name 'summarizer' is not defined